In [1]:
import pandas as pd

orders_df = pd.read_csv("orders.csv")
print(orders_df.head())


   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [3]:
users_df = pd.read_json("users.json")
print(users_df.head())


   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [5]:
import sqlite3

# Create SQLite DB
conn = sqlite3.connect("restaurants.db")

# Load SQL file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)

restaurants_df = pd.read_sql_query(
    "SELECT * FROM restaurants", conn
)

print(restaurants_df.head())


   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [7]:
# Merge orders + users
orders_users_df = pd.merge(
    orders_df,
    users_df,
    on="user_id",
    how="left"
)

# Merge with restaurants
final_df = pd.merge(
    orders_users_df,
    restaurants_df,
    on="restaurant_id",
    how="left"
)


In [9]:
final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print("Final dataset saved successfully!")


Final dataset saved successfully!


In [12]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])
final_df.groupby(final_df['order_date'].dt.month)['order_id'].count()


C:\Users\hp\AppData\Local\Temp\ipykernel_4512\2050272084.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


order_date
1     831
2     785
3     903
4     812
5     844
6     784
7     859
8     851
9     812
10    863
11    807
12    849
Name: order_id, dtype: int64

In [17]:
final_df.groupby('user_id')['total_amount'].mean()


user_id
1       1289.660000
2        756.412000
3        919.755000
4        935.290000
5       1148.576000
           ...     
2996     511.180000
2997    1062.064000
2998     706.911667
2999     643.890000
3000    1113.315000
Name: total_amount, Length: 2883, dtype: float64

In [21]:
final_df.groupby('city')['total_amount'].sum()
final_df.groupby('cuisine')['order_id'].count()


cuisine
Chinese    2418
Indian     2469
Italian    2532
Mexican    2581
Name: order_id, dtype: int64

In [28]:
final_df.groupby('membership')['total_amount'].mean()


membership
Gold       797.145556
Regular    805.158434
Name: total_amount, dtype: float64

In [33]:
final_df.groupby(final_df['order_date'].dt.month)['total_amount'].sum()


order_date
1     663020.26
2     630867.40
3     716738.98
4     666097.18
5     668428.61
6     610822.93
7     688559.45
8     694987.58
9     653838.07
10    699187.73
11    637772.43
12    681303.50
Name: total_amount, dtype: float64

In [39]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset (1).csv")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [43]:
df[df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [45]:
df.groupby('cuisine')['total_amount'] \
  .mean() \
  .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [47]:
user_spend = df.groupby('user_id')['total_amount'].sum()

user_spend[user_spend > 1000].count()


2544

In [53]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

df['rating_range'] = pd.cut(
    df['rating'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

df.groupby('rating_range')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


C:\Users\hp\AppData\Local\Temp\ipykernel_4512\3419888468.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('rating_range')['total_amount'] \


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [55]:
df[df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [57]:
restaurant_count = df.groupby('cuisine')['restaurant_id'].nunique()
revenue = df.groupby('cuisine')['total_amount'].sum()

pd.concat(
    [restaurant_count, revenue],
    axis=1,
    keys=['restaurant_count', 'total_revenue']
).sort_values(
    by=['restaurant_count', 'total_revenue'],
    ascending=[True, False]
)


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [61]:
gold_orders = df[df['membership'] == 'Gold'].shape[0]
total_orders = df.shape[0]

round((gold_orders / total_orders) * 100)


50

In [66]:
restaurant_stats = df.groupby('restaurant_name_x').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
).reset_index()

restaurant_stats[restaurant_stats['total_orders'] < 20] \
    .sort_values('avg_order_value', ascending=False)


,restaurant_name_x,avg_order_value,total_orders
173,Hotel Dhaba Multicuisine,1040.222308,13
354,Sri Mess Punjabi,1029.180833,12
236,Ruchi Biryani Punjabi,1002.140625,16
343,Sri Delights Pure Veg,989.467222,18
88,Classic Kitchen Family Restaurant,973.167895,19
...,...,...,...
64,Annapurna Tiffins Punjabi,621.828947,19
126,Darbar Tiffins Non-Veg,596.815556,18
122,Darbar Restaurant Punjabi,589.972857,14
333,Spice Tiffins Pure Veg,578.578667,15


In [70]:
df.groupby(['membership', 'cuisine'])['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [72]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['quarter'] = df['order_date'].dt.quarter

df.groupby('quarter')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


C:\Users\hp\AppData\Local\Temp\ipykernel_4512\825624610.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [74]:
gold_orders_count = df[df['membership'] == 'Gold'].shape[0]
gold_orders_count


4987

In [76]:
hyderabad_revenue = df[df['city'] == 'Hyderabad']['total_amount'].sum()
round(hyderabad_revenue)


1889367

In [78]:
df['user_id'].nunique()


2883

In [82]:
gold_avg_order_value = df[df['membership'] == 'Gold']['total_amount'].mean()
round(gold_avg_order_value, 2)


797.15

In [86]:
df[df['rating'] >= 4.5].shape[0]


3374

In [90]:
df[
    (df['membership'] == 'Gold') &
    (df['city'] == 'Chennai')
].shape[0]


1337